## initialization

### imports

In [1]:
from initailFunctionsPath import *
PATH_TRADE = "/home/user1/Data/"
PATH_PORTFOLIO = "/home/user1/Data/Portfolio/{}"
PRICE_PATH = "/home/user1/Data/{}"
VALID_SYMBOLS_PATH = "/home/user1/Data/"
PATH_OUTPUT = "/home/user1/Data/Outputs/"

### Spark instaniation

In [2]:
conf = SparkConf()
(conf
.set('spark.driver.memory', '130g')
.set('spark.executer.cores', '58')
.set('spark.shuffle.service.index.cache.size', '1g')
.setAppName('Practice') )
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

22/03/07 09:47:47 WARN Utils: Your hostname, user1-ubuntu resolves to a loopback address: 127.0.1.1; using 172.16.32.107 instead (on interface eth0)
22/03/07 09:47:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/07 09:47:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## data inputs

### load daily price and shrout data

In [3]:
price_df = (
    spark.read.parquet(PRICE_PATH.format('Cleaned_Stock_Prices_14001127.parquet'))
    .filter(F.col('jalaliDate').between(MIN_ANALYSIS_DATE, MAX_ANALYSIS_DATE))
    .select(
        F.col('jalaliDate').alias('date'),
        F.col('name').alias('symbol'),
        'close_price',
        'close_price_adjusted',
        'shrout',
        (F.col('MarketCap') / 10**7).alias('mktcap')
    )
    .dropDuplicates()
)

price_df = replace_arabic_characters_and_correct_symbol_names(price_df)

display_df(price_df)

22/03/07 09:48:12 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


71822


22/03/07 09:48:41 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


+--------+------+-----------+--------------------+------+---------+
|date    |symbol|close_price|close_price_adjusted|shrout|mktcap   |
+--------+------+-----------+--------------------+------+---------+
|13980221|آ س پ |1366.0     |1249.0              |9.0E8 |122940.0 |
|13980125|آتیمس |30009.0    |30380.0             |1.0E9 |3000900.0|
|13980202|آتیمس |30880.0    |31262.0             |1.0E9 |3088000.0|
+--------+------+-----------+--------------------+------+---------+
only showing top 3 rows



In [4]:
min_max(price_df)

22/03/07 09:48:42 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


+--------+--------+
|min_date|max_date|
+--------+--------+
|13980105|13980329|
+--------+--------+



In [5]:
MIN_PRICE_DATE = price_df.agg(F.min('date')).collect()[0][0]
MAX_PRICE_DATE = price_df.agg(F.max('date')).collect()[0][0]

price_df.agg(F.countDistinct('symbol')).show()
price_df.filter(F.col('date') == MIN_PRICE_DATE).agg(F.countDistinct('symbol')).show()
price_df.filter(F.col('date') == MAX_PRICE_DATE).agg(F.countDistinct('symbol')).show()

22/03/07 09:48:46 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:48:49 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:48:52 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:49:00 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


+-------------+
|count(symbol)|
+-------------+
|         1329|
+-------------+



22/03/07 09:49:01 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:49:04 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


+-------------+
|count(symbol)|
+-------------+
|         1252|
+-------------+



22/03/07 09:49:05 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:49:08 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


+-------------+
|count(symbol)|
+-------------+
|         1309|
+-------------+



### load valid symbols data

In [6]:
valid_symbols_df = (
    spark.read.parquet(VALID_SYMBOLS_PATH + '{}'.format('validSymbols.parquet'))
)

display_df(valid_symbols_df)

1284
+------+
|symbol|
+------+
|ختوقا |
|همراه |
|بالاس |
+------+
only showing top 3 rows



## data preparation

### prepare trade data

In [7]:
trade_df = (
    spark.read.parquet(PATH_TRADE + "{}".format("trade_df.parquet"))
)
display_df(trade_df)
# Note: 'time' columns is not reliable!

26223382
+--------+------+----------------+------+------------+----------+--------------------+------------------------------------+------------------------------------+
|date    |time  |secondsWithinDay|symbol|nTradeShares|tradePrice|tradeSettlementValue|buyerAccountId                      |sellerAccountId                     |
+--------+------+----------------+------+------------+----------+--------------------+------------------------------------+------------------------------------+
|13980110|122306|12963606        |ثاباد |500         |2297.0    |0.11485             |993C6DD6-49B4-47F3-A893-9064F762C446|D1A83C66-FC1F-4D5B-8A3D-CB925246DC1A|
|13980110|94035 |12963635        |ثاباد |8000        |2321.0    |1.8568              |EB3B2010-24BA-4E2B-A438-58BEB542542D|DF9E0BAC-E4EF-450F-A5AA-1F62CF9A2A5E|
|13980110|92919 |12963619        |ثاباد |500         |2290.0    |0.1145              |8AFD82A0-75A6-4187-8CF9-D6A6B9A99978|81D61F41-D929-4E4E-A721-6A5AC2C3336D|
+--------+------+--------

In [8]:
print('missing nTradeShares: ', round(trade_df.filter(F.col('nTradeShares') == 0).count() / trade_df.count(), 5))
print('missing tradeSettlementValue: ', round(trade_df.filter(F.col('tradeSettlementValue') == 0).count() / trade_df.count(), 5))

missing nTradeShares:  0.0
missing tradeSettlementValue:  0.0


### prepare initial portfolio data

In [9]:
portfolio_df = spark.read.parquet(
    PATH_PORTFOLIO + "{}".format("portfolio_df.parquet")
    )
display_df(portfolio_df)

11503599
+------------------------------------+--------+------+-----------+
|accountId                           |date    |symbol|nHeldShares|
+------------------------------------+--------+------+-----------+
|9F4FDECE-0613-4162-8EFB-87A683771C14|13980105|خپارس |53000      |
|6BD95B55-3219-4F17-9EEA-09A450C7ADB8|13980105|ونفت  |496        |
|34149238-6FE5-4291-9518-4C188BA3B22B|13980105|وخارزم|18626      |
+------------------------------------+--------+------+-----------+
only showing top 3 rows



In [10]:
portfolio_df = (
    portfolio_df
    .join(valid_symbols_df, on = ['symbol'], how = 'inner')
)

display_df(portfolio_df)

8807831
+------+------------------------------------+--------+-----------+
|symbol|accountId                           |date    |nHeldShares|
+------+------------------------------------+--------+-----------+
|خپارس |9F4FDECE-0613-4162-8EFB-87A683771C14|13980105|53000      |
|ونفت  |6BD95B55-3219-4F17-9EEA-09A450C7ADB8|13980105|496        |
|وخارزم|34149238-6FE5-4291-9518-4C188BA3B22B|13980105|18626      |
+------+------------------------------------+--------+-----------+
only showing top 3 rows



In [11]:
(
    portfolio_df
    .filter(F.col('nHeldShares') < 0)
    .count()
)

0

#### check symbols

In [12]:
price_symbols = price_df.select('symbol').distinct().withColumn('price', F.lit(1))
trade_symbols = trade_df.select('symbol').distinct().withColumn('trade', F.lit(1))
portfolio_symbols = portfolio_df.select('symbol').distinct().withColumn('portfolio', F.lit(1))

symbols_df = (
    trade_symbols
    .join(portfolio_symbols, on = ['symbol'], how = 'outer')
    .join(price_symbols, on = ['symbol'], how = 'outer')
)

print(symbols_df.filter(F.col('price').isNull()).select('symbol').rdd.flatMap(lambda x: x).collect())

22/03/07 09:51:37 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:51:47 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


['نشار']


In [13]:
w = Window().partitionBy('symbol').orderBy('date')
w2 = Window().partitionBy('symbol').orderBy(price_df.date.desc())

price_return_df = (price_df.select('symbol', 'mktcap',
   ((
       F.first('close_price',True).over(w2) - F.first('close_price',True).over(w)
       ) 
       / F.first('close_price',True).over(w)
       ).alias('price_return'),
       )
   .dropDuplicates(['symbol'])
)
tempt = price_df.na.drop(
    how = 'any',
)
large_small_stocks =( 
    tempt[tempt.date == 13980328]
    .withColumn('sizeDecile', F.ntile(10).over(Window.partitionBy().orderBy('mktcap')))
) 
price_return_df = price_return_df.join(
    large_small_stocks.select(
        F.col('symbol'),
    F.col('sizeDecile'),
    ) , on =['symbol']
).select(
    F.col('symbol'),
    F.col('mktcap'),
    F.col('price_return'),
    F.col('sizeDecile'),
)

In [14]:
(
    price_return_df
    .groupBy('sizeDecile')
    .agg(
        F.round(F.expr('percentile(mktcap, array(0.5))')[0], 3).alias('medianmktcap'),
        F.round(F.mean('price_return'), 2).alias('meanReturn'),
        F.round(F.expr('percentile(price_return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .orderBy('sizeDecile')
    .show()
)

22/03/07 09:52:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/03/07 09:52:07 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:52:07 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/03/07 09:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/03/07 09:52:15 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:52:16 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB
22/03/07 09:52:17 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


+----------+------------+----------+------------+
|sizeDecile|medianmktcap|meanReturn|medianReturn|
+----------+------------+----------+------------+
|         1|         0.0|      0.01|         0.0|
|         2|         0.0|       0.0|         0.0|
|         3|         0.0|   1362.94|         0.0|
|         4|   21193.443|      3.85|         0.0|
|         5|   59854.825|      0.55|         0.0|
|         6|    115544.0|      0.53|       0.389|
|         7|   203169.76|      0.57|       0.371|
|         8|    342523.7|      0.58|        0.46|
|         9|   731217.75|      0.55|       0.404|
|        10|   2902200.0|      0.33|       0.228|
+----------+------------+----------+------------+



### general insights

#### check compatibility of the two datasets

In [15]:
common_investors_df = (
    trade_df
    .select(F.col('buyerAccountId').alias('accountId'))
    .union(trade_df.select(F.col('sellerAccountId').alias('accountId')))
    .dropDuplicates()
    .withColumn('trade', F.lit(1))
    .join(portfolio_df.select('accountId', F.lit(1).alias('portfolio')).dropDuplicates(), on = ['accountId'], how = 'outer')
    .fillna(0, subset = ['trade', 'portfolio'])
)

display_df(common_investors_df)

4382954
+------------------------------------+-----+---------+
|accountId                           |trade|portfolio|
+------------------------------------+-----+---------+
|00019AFD-C89B-4B63-98BB-18BF5A112C6F|0    |1        |
|000217BA-3C48-4458-8CAA-691CA19C7187|0    |1        |
|000249D5-649B-4C99-AE9E-82AB979E80C9|0    |1        |
+------------------------------------+-----+---------+
only showing top 3 rows



In [16]:
trade_only = common_investors_df.filter( (F.col('trade') == 1) & (F.col('portfolio') == 0)).count()
all_trade = common_investors_df.filter(F.col('trade') == 1).count()

print('share of missing portfolio accounts among traders:', round(100 * trade_only / all_trade, 2), '%')
# It seems reasonable to attribute this missing portion to the new entrants!

share of missing portfolio accounts among traders: 21.06 %


In [17]:
portfolio_only = common_investors_df.filter( (F.col('trade') == 0) & (F.col('portfolio') == 1)).count()
all_portfolio = common_investors_df.filter(F.col('portfolio') == 1).count()

print('share of missing trades among investors who have nitial portfolio:', round(100 * portfolio_only / all_portfolio, 2), '%')
# It seems reasonable to attribute this missing portion to the new entrants!

share of missing trades among investors who have nitial portfolio: 86.23 %


#### number of unique investors

In [18]:
(
    trade_df
    .select(F.col('buyerAccountId').alias('accountId'))
    .union(trade_df.select(F.col('sellerAccountId').alias('accountId')))
    .dropDuplicates()
    .count()
)

737297

#### number of stocks within investors' initial portfolios

In [19]:
(
    portfolio_df
    .groupBy('accountId')
    .count()
    .agg(
        F.expr('percentile(count, array(0.25))')[0].alias('25%'),
        F.expr('percentile(count, array(0.50))')[0].alias('50%'),
        F.round(F.mean('count'), 4).alias('mean'),
        F.expr('percentile(count, array(0.75))')[0].alias('75%'),
        F.expr('percentile(count, array(0.9))')[0].alias('90%'),
        F.expr('percentile(count, array(0.99))')[0].alias('99%'),
    )
    .show()
)

+---+---+------+---+---+----+
|25%|50%|  mean|75%|90%| 99%|
+---+---+------+---+---+----+
|1.0|1.0|2.0834|2.0|4.0|15.0|
+---+---+------+---+---+----+



#### compare trade value of new entrants with other investors

In [20]:
(
    common_investors_df
    .select(F.col('accountId').alias('buyerAccountId'), F.col('portfolio').alias('hasPortfolio'))
    .join(trade_df, on = ['buyerAccountId'], how = 'right')
    .groupBy('hasPortfolio')
    .agg(
        F.round(F.expr('percentile(tradeSettlementValue, array(0.5))')[0], 2).alias('median_buyValue'),
        F.round(F.mean('tradeSettlementValue'), 2).alias('mean_buyValue')
    )
    .show()
)

+------------+---------------+-------------+
|hasPortfolio|median_buyValue|mean_buyValue|
+------------+---------------+-------------+
|           1|            1.0|         3.44|
|           0|           0.78|         2.57|
+------------+---------------+-------------+



#### stocks whose shares were given to the mass general public

In [21]:
mass_public_stocks_df = (
    portfolio_df
    .groupBy('symbol', 'nHeldShares')
    .agg(
        F.countDistinct('accountId').alias('nHolders')
    )
    .withColumn('nAllHolders', F.sum('nHolders').over(Window.partitionBy('symbol')))
    .withColumn('rank', F.row_number().over(Window.partitionBy('symbol').orderBy(F.desc('nHolders'))))
    .filter(F.col('rank') == 1)
    .drop('rank')
    .orderBy(F.desc('nHolders'))
    .withColumn('shareOfHolders', F.round(F.col('nHolders') / F.col('nAllHolders'), 3))
#     .join(price_df.filter(F.col('date') == 13980105).select('symbol', 'shrout'), on = 'symbol', how = 'left')
#     .withColumn('shareOfShares', F.round(F.col('nHeldShares')*F.col('nHolders') / F.col('shrout'), 3))
)

display_df(mass_public_stocks_df)

665
+------+-----------+--------+-----------+--------------+
|symbol|nHeldShares|nHolders|nAllHolders|shareOfHolders|
+------+-----------+--------+-----------+--------------+
|زماهان|120        |266735  |332130     |0.803         |
|ومعلم |229        |209919  |261378     |0.803         |
|سمایه |300        |185411  |302045     |0.614         |
+------+-----------+--------+-----------+--------------+
only showing top 3 rows



In [22]:
mass_public_stocks_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'mass_public_stocks.parquet')

## make daily portfolios

### flatten trade data

In [23]:
raw_flat_trade_df = spark.read.parquet(PATH_TRADE + "{}".format("raw_flat_trade_df.parquet"))

display_df(raw_flat_trade_df)

15170298
+--------+------+------------------------------------+------------+---------+------+
|date    |symbol|accountId                           |nTradeShares|cashOut  |cashIn|
+--------+------+------------------------------------+------------+---------+------+
|13980204|ومعلم |D41B1B0F-FB40-4ACE-BF3B-4AA4E6700EA2|-229        |0.074654 |0.0   |
|13980204|پارس  |D41B1B0F-FB40-4ACE-BF3B-4AA4E6700EA2|-2          |0.0076702|0.0   |
|13980204|بزاگرس|D41B1B0F-FB40-4ACE-BF3B-4AA4E6700EA2|-449        |0.145476 |0.0   |
+--------+------+------------------------------------+------------+---------+------+
only showing top 3 rows



In [24]:
print(round(100*raw_flat_trade_df.filter(
    (F.col('cashIn') != 0)&
    (F.col('cashOut') != 0) ).count() / raw_flat_trade_df.count(), 2),'%')

3.42 %


In [25]:
print(raw_flat_trade_df.filter(F.col('nTradeShares') == 0).count())
print(trade_df.filter(F.col('tradeSettlementValue') == 0).count())

187070
0


In [26]:
print(raw_flat_trade_df.filter(F.col('cashIn') > 0 ).count())
print(raw_flat_trade_df.filter(F.col('cashOut') < 0 ).count())

0
0


### make daily portfolios

In [27]:
adjustment_ratio = price_df.withColumn(
    "ratio", F.col('close_price_adjusted') / F.col('close_price') 
).select(
    F.col('date'),
    F.col('symbol'),
    F.col('ratio'),
)

In [28]:
adjusted_portfolio_df =( 
    portfolio_df.join(
    adjustment_ratio,
    on = ['date','symbol']
    )
    .fillna(1)
    .withColumn("nHeldShares",F.col("nHeldShares") / F.col('ratio'))
    .drop('ratio')
)
display_df(
    adjusted_portfolio_df
    )

22/03/07 09:57:51 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


8918727
+--------+------+------------------------------------+------------------+
|date    |symbol|accountId                           |nHeldShares       |
+--------+------+------------------------------------+------------------+
|13980105|خپارس |9F4FDECE-0613-4162-8EFB-87A683771C14|266292.68292682926|
|13980105|ونفت  |6BD95B55-3219-4F17-9EEA-09A450C7ADB8|1700.1546218487395|
|13980105|وخارزم|34149238-6FE5-4291-9518-4C188BA3B22B|29695.876460767944|
+--------+------+------------------------------------+------------------+
only showing top 3 rows



In [29]:
adjusted_raw_flat_trade_df = (
    raw_flat_trade_df.join(
    adjustment_ratio,
    on = ['date','symbol']
    )
    .withColumn("nTradeShares",F.col("nTradeShares") / F.col('ratio'))
    .drop('ratio')
)
display_df(
    adjusted_raw_flat_trade_df
    )

22/03/07 09:58:52 WARN DAGScheduler: Broadcasting large task binary with size 17.9 MiB


15197300
+--------+------+------------------------------------+-------------------+---------+------+
|date    |symbol|accountId                           |nTradeShares       |cashOut  |cashIn|
+--------+------+------------------------------------+-------------------+---------+------+
|13980204|ومعلم |D41B1B0F-FB40-4ACE-BF3B-4AA4E6700EA2|-701.9673202614379 |0.074654 |0.0   |
|13980204|پارس  |D41B1B0F-FB40-4ACE-BF3B-4AA4E6700EA2|-2.9215262778977684|0.0076702|0.0   |
|13980204|بزاگرس|D41B1B0F-FB40-4ACE-BF3B-4AA4E6700EA2|-924.8315321042594 |0.145476 |0.0   |
+--------+------+------------------------------------+-------------------+---------+------+
only showing top 3 rows



In [30]:
raw_daily_portfolio_df = (
    adjusted_portfolio_df
    # portfolio_df
    .select('date',
            'symbol', 
            'accountId', 
            'nHeldShares', 
            F.lit(0).alias('cashOut'),
            F.lit(0).alias('cashIn')
           )
    .union(
        adjusted_raw_flat_trade_df
        # raw_flat_trade_df
        .withColumnRenamed('nTradeShares', 'nHeldShares')
    )
    .groupBy('date', 'symbol', 'accountId')
    .agg(
        F.sum('nHeldShares').alias('nHeldShares'),
        F.sum('cashOut').alias('cashOut'),
        F.sum('cashIn').alias('cashIn')
    )
    .orderBy('accountId', 'date')
    .withColumn('heldShares', make_daily_portfolio()[0])
    .withColumn('netCashOut', make_daily_portfolio()[1])
    .withColumn('netCashIn', make_daily_portfolio()[2])
    .drop('nHeldShares', 'settlementValue', 'cashIn', 'cashOut')
)

display_df(raw_daily_portfolio_df)

23803740
+--------+------+------------------------------------+-------------------+----------+---------+
|date    |symbol|accountId                           |heldShares         |netCashOut|netCashIn|
+--------+------+------------------------------------+-------------------+----------+---------+
|13980105|چکاپا |00026661-733B-49E0-AC93-ED5812290A9B|6951.577928363989  |0.0       |0.0      |
|13980221|چکاپا |00026661-733B-49E0-AC93-ED5812290A9B|-2.6461380260525402|0.83346   |0.0      |
|13980105|ارفع  |00029878-6EF9-49A7-B231-61DBED05BDE7|86936.4768425292   |0.0       |0.0      |
+--------+------+------------------------------------+-------------------+----------+---------+
only showing top 3 rows



#### invalid holdings

In [31]:
invalid_holdings_df = (
    raw_daily_portfolio_df
    .filter(F.col('heldShares') < 0)
    .select('accountId', 'symbol')
    .dropDuplicates()
    .withColumn('invalidHolding', F.lit(1))
)
display_df(invalid_holdings_df)


flat_trade_df = (
    adjusted_raw_flat_trade_df
    .join(invalid_holdings_df, on = ['accountId', 'symbol'], how = 'left')
    .filter(F.col('invalidHolding').isNull())
    .drop('invalidHolding')
)
display_df(flat_trade_df)


daily_portfolio_df = (
    raw_daily_portfolio_df
    .join(invalid_holdings_df, on = ['accountId', 'symbol'], how = 'left')
    .filter(F.col('invalidHolding').isNull())
    .drop('invalidHolding')
)
display_df(daily_portfolio_df)

1722366
+------------------------------------+------+--------------+
|accountId                           |symbol|invalidHolding|
+------------------------------------+------+--------------+
|00026661-733B-49E0-AC93-ED5812290A9B|چکاپا |1             |
|000A5967-88E3-4C5D-8689-58BFBCA82C0A|خپارس |1             |
|001211AB-93FF-4B78-ACB2-845909058F1B|فباهنر|1             |
+------------------------------------+------+--------------+
only showing top 3 rows



10733738
+------------------------------------+------+--------+------------------+---------+--------+
|accountId                           |symbol|date    |nTradeShares      |cashOut  |cashIn  |
+------------------------------------+------+--------+------------------+---------+--------+
|00067589-1512-4653-8AC5-A6354E0957AD|خساپا |13980209|12461.68582375479 |0.0      |-0.32875|
|0007778B-EBC1-4864-A598-4D18159B683E|ومعلم |13980222|-701.7522299306244|0.0704633|0.0     |
|0007F5EF-A1DC-4156-A80F-FCD9930A3689|زشریف |13980231|663.4090909090909 |0.0      |-0.05838|
+------------------------------------+------+--------+------------------+---------+--------+
only showing top 3 rows



18755774
+------------------------------------+------+--------+------------------+----------+---------+
|accountId                           |symbol|date    |heldShares        |netCashOut|netCashIn|
+------------------------------------+------+--------+------------------+----------+---------+
|00029878-6EF9-49A7-B231-61DBED05BDE7|ارفع  |13980105|86936.4768425292  |0.0       |0.0      |
|0002A50F-2B48-42F6-B586-428F3BB6423A|کرمان |13980105|24880.324675324675|0.0       |0.0      |
|00067589-1512-4653-8AC5-A6354E0957AD|خساپا |13980209|12461.68582375479 |0.0       |-0.32875 |
+------------------------------------+------+--------+------------------+----------+---------+
only showing top 3 rows



In [ ]:
daily_portfolio_df..write.mode('overwrite').parquet(PATH_PORTFOLIO + 'daily_portfolio_df.parquet')

#### new entrants df

In [ ]:
new_entrant_account_ids_df = (
    flat_trade_df
    .groupBy('accountId')
    .agg(
        F.min('date').alias('firstDate')
    )
    .join(portfolio_df.select('accountId').distinct(), on = 'accountId', how = 'left_anti')
)

display_df(new_entrant_account_ids_df)

In [ ]:
new_entrant_account_ids_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'new_entrant_account_ids.parquet')

#### time series of new entrants

In [ ]:
new_entrantd_time_series_df = (
    flat_trade_df
    .select('date', 'accountId')
    .dropDuplicates()
    .join(new_entrant_account_ids_df, on = 'accountId', how = 'inner')
    .withColumn('rank', F.row_number().over(Window.partitionBy('accountId').orderBy('date')))
    .filter(F.col('rank') == 1)
    .drop('rank')
    .groupBy('date')
    .count()
    .orderBy('date')
)

display_df(new_entrantd_time_series_df)

In [ ]:
new_entrantd_time_series_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'new_entrantd_time_series_df.parquet')

### calculate gain from trade

In [ ]:
gain_from_trade_df = (
    flat_trade_df
    .groupBy('accountId')
    .agg(
        F.sum('cashOut').alias('netCashOut'),
        F.sum('cashIn').alias('netCashIn'),
    )
)

display_df(gain_from_trade_df)

### calculate value of the initial portfolio

In [ ]:
initial_portfolio_value_df = (
    adjusted_portfolio_df
    .join(price_df.select('date', 'symbol', 'close_price_adjusted'), on = ['date', 'symbol'], how = 'left')
    .withColumnRenamed("close_price_adjusted","close_price")
    .dropna(subset = ['close_price'])
    .join(invalid_holdings_df, on = ['accountId', 'symbol'], how = 'left')
    .filter(F.col('invalidHolding').isNull())
    .withColumn('value', F.col('nHeldShares') * F.col('close_price'))
    .groupBy('accountId')
    .agg(
        (F.sum('value') / 10**7).alias('initialPortfolioValue')
    )
)

display_df(initial_portfolio_value_df)
# count after join?

In [ ]:
print(initial_portfolio_value_df.filter(F.col('initialPortfolioValue').isNull()).count())
print(initial_portfolio_value_df.filter(F.col('initialPortfolioValue') <= 0).count())

### calculate value of the final portfolio

In [ ]:
final_portfolio_value_df = (
    daily_portfolio_df
    .withColumn('rowNumber', F.row_number().over(Window.partitionBy('accountId', 'symbol').orderBy('date')))
    .withColumn('maxRowNumber', F.max('rowNumber').over(Window.partitionBy('accountId', 'symbol')))
    .filter(F.col('rowNumber') == F.col('maxRowNumber'))
    .filter(F.col('heldShares') > 0)
    .withColumn('date', F.lit(MAX_PRICE_DATE))
    .join(price_df.select('date', 'symbol', 'close_price_adjusted'), on = ['date', 'symbol'], how = 'left')
    .withColumnRenamed("close_price_adjusted","close_price")
    .dropna(subset = ['close_price'])
    .withColumn('value', F.col('heldShares') * F.col('close_price'))
    .groupBy('accountId')
    .agg(
        (F.sum('value') / 10**7).alias('finalPortfolioValue')
    )   
)

display_df(final_portfolio_value_df)
# count after join?

In [ ]:
print(final_portfolio_value_df.filter(F.col('finalPortfolioValue').isNull()).count())
print(final_portfolio_value_df.filter(F.col('finalPortfolioValue') <= 0).count())

In [ ]:
t = final_portfolio_value_df.select("finalPortfolioValue").rdd.max()[0]
final_portfolio_value_df.filter(F.col('finalPortfolioValue') == t).show(1,False)

## Find symbols in portfolios with one symbol

In [ ]:
stock_number_df = ( daily_portfolio_df
.dropDuplicates(subset= ['accountId','symbol'])
.withColumn('numberOfStocks', F.row_number().over(Window.partitionBy('accountId').orderBy(F.desc('heldShares'))))
.withColumn('numberOfStocks', F.max('numberOfStocks').over(Window.partitionBy('accountId')))
)
display_df(stock_number_df)
    

In [ ]:
symbols_in_one_symbol_portfolios_df = (
    stock_number_df.filter(F.col('numberOfStocks') == 1)
    .withColumn("accountNumbers", F.row_number().over(Window.partitionBy('symbol').orderBy(F.desc('heldShares'))))
    .withColumn("accountNumbers", F.max('accountNumbers').over(Window.partitionBy('symbol')))
    .dropDuplicates(subset = ['symbol'])
    .select(
        F.col('symbol'),
        F.col('accountNumbers'),
    )
    .orderBy(F.desc('accountNumbers'))
)
display_df(symbols_in_one_symbol_portfolios_df)

In [ ]:
symbols_in_one_symbol_portfolios_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'symbols_in_one_symbol_portfolios_df.parquet')

### time series of the net cash in

In [ ]:
max_portfolio_value_df = (
    final_portfolio_value_df
    .join(initial_portfolio_value_df, on = ['accountId'], how = 'outer')
    .fillna(0)
    .withColumn('maxPortfolioValue', F.greatest(F.col('initialPortfolioValue'), F.col('finalPortfolioValue')))
    .withColumn('type', F.when(F.col('maxPortfolioValue') < 10, 'lessThan10MT')
                         .when(F.col('maxPortfolioValue').between(10, 20), 'between10MTand20MT')
                         .when(F.col('maxPortfolioValue').between(20, 50), 'between20MTand50MT')
                         .otherwise('greaterThan50MT')
               )
    .select('accountId', 'type')
    .dropDuplicates()
)

display_df(max_portfolio_value_df)

In [ ]:
dates_list = price_df.select('date').distinct().orderBy('date').rdd.flatMap(lambda x: x).collect()

cash_time_series_df = (
    flat_trade_df
    .withColumn('netCash', F.col('cashIn') + F.col('cashOut'))
    .join(max_portfolio_value_df, on = 'accountId', how = 'inner')
    .groupBy('type', 'date')
    .agg(
        F.round((-F.sum('netCash'))).alias('netCash'),
        F.countDistinct('accountId').alias('nAccounts')
    )
    .orderBy('date', 'type')
)

cash_time_series_df.show(4)

In [ ]:
cash_time_series_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'cash_time_series.parquet')

## calculate returns

In [ ]:
return_df = (
    gain_from_trade_df
    .join(initial_portfolio_value_df, on = 'accountId', how = 'outer')
    .join(final_portfolio_value_df, on = 'accountId', how = 'outer')
    .fillna(0, subset = ['netCashIn', 'netCashOut', 'initialPortfolioValue', 'finalPortfolioValue'])
    .withColumn('return', 
                ((F.col('finalPortfolioValue') + F.col('netCashOut')) / (F.col('initialPortfolioValue') + (-F.col('netCashIn')))) - 1)
    .filter(F.col('return').isNotNull())
    .withColumn('returnDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('return')))
)

display_df(return_df)
# null returns?

In [ ]:
(
    return_df
    .filter(F.col('return') == 0)
    .count()
)


In [ ]:
(
    return_df
    .groupBy('returnDecile')
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .show()
)

In [ ]:
(
    return_df
    .agg(
       F.round(F.min('return'), 2).alias('min'),
        F.round(F.expr('percentile(return, array(0.01))')[0], 2).alias('1%'),
        F.round(F.expr('percentile(return, array(0.1))')[0], 2).alias('10%'),
        F.round(F.expr('percentile(return, array(0.25))')[0], 2).alias('25%'),
        F.round(F.expr('percentile(return, array(0.5))')[0], 2).alias('50%'),
        F.round(F.mean('return'), 2).alias('mean'),
        F.round(F.expr('percentile(return, array(0.75))')[0], 2).alias('75%'),
        F.round(F.expr('percentile(return, array(0.9))')[0], 2).alias('90%'),
        F.round(F.expr('percentile(return, array(0.99))')[0], 2).alias('99%'),
        F.round(F.expr('percentile(return, array(0.999))')[0], 2).alias('99.9%'),
    )
    .show()
)

In [ ]:
return_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'return_output.parquet')

In [ ]:
return_df.filter(F.col("return") == return_df.select("return").rdd.max()[0]).show(20,False)


In [ ]:
portfolio_df.filter(F.col("accountId") == 'D832B9B2-00EB-489E-8B91-8C1BF4BE8B26').show(2,False)

In [ ]:
(
    price_df
    .filter((F.col("symbol") == 'ودانا')&(F.col("date") > 13980105))
    .orderBy("date")
    .select(
        F.col("date"),
        F.col("close_price"),
        F.col("close_price_adjusted"),
    )
    ).show(20,False)

In [ ]:
final_portfolio_value_df.filter(F.col("accountId") == 'D832B9B2-00EB-489E-8B91-8C1BF4BE8B26').show(2,False)
initial_portfolio_value_df.filter(F.col("accountId") == 'D832B9B2-00EB-489E-8B91-8C1BF4BE8B26').show(2,False)

### final portfolio value output

In [ ]:
output_final_portfolio_value = (
    final_portfolio_value_df
    .join(return_df.select('accountId', 'return'), on = 'accountId')
    .withColumn('finalPortfolioValueDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('finalPortfolioValue')))
)

display_df(output_final_portfolio_value)

In [ ]:
(
    output_final_portfolio_value
    .groupBy('finalPortfolioValueDecile')
    .agg(
        F.round(F.expr('percentile(finalPortfolioValue, array(0.5))')[0], 3).alias('medianFinalPortfolioValue'),
        F.round(F.mean('return'), 2).alias('meanReturn'),
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .orderBy('finalPortfolioValueDecile')
    .show()
)

In [ ]:
output_final_portfolio_value.write.mode('overwrite').parquet(PATH_OUTPUT + 'final_portfolio_output.parquet')

### initial portfolio value output

In [ ]:
output_initial_portfolio_value = (
    initial_portfolio_value_df
    .join(return_df.select('accountId', 'return'), on = 'accountId')
    .withColumn('initialPortfolioValueDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('initialPortfolioValue')))
)

display_df(output_initial_portfolio_value)

In [ ]:
(
    output_initial_portfolio_value
    .groupBy('initialPortfolioValueDecile')
    .agg(
        F.round(F.expr('percentile(initialPortfolioValue, array(0.5))')[0], 3).alias('medianInitialPortfolioValue'),
        F.round(F.mean('return'), 2).alias('meanReturn'),
        F.round(F.expr('percentile(return, array(0.5))')[0], 5).alias('medianReturn')
    )
    .orderBy('initialPortfolioValueDecile')
    .show()
)

In [ ]:
output_initial_portfolio_value.write.mode('overwrite').parquet(PATH_OUTPUT + 'inital_portfolio_output.parquet')

### calculate frequency of trades and active days

In [ ]:
active_days_df = (
    adjusted_raw_flat_trade_df
    .groupBy('accountId', 'date')
    .agg(
        F.sum('cashIn').alias('netCashIn'),
        F.sum('cashOut').alias('netCashOut')
    )
    .withColumn('netCash', F.col('netCashIn') + F.col('netCashOut'))
    .groupBy('accountId')
    .agg(
        F.count(F.when(F.col('netCash') < 0, F.lit(1))).alias('nBuyDays'),
        F.count(F.when(F.col('netCash') > 0, F.lit(1))).alias('nSellDays')
    )
    .fillna(0, subset = ['nBuyDays', 'nSellDays'])
)

display_df(active_days_df)

In [ ]:
buy_trade_df = (
    trade_df
        .select(
        'date',
        'symbol',
        F.col('buyerAccountId').alias('accountId'),
        'nTradeShares',
        (-F.col('tradeSettlementValue')).alias('settlementValue'),
        )
)

sell_trade_df = (
    trade_df
        .select(
            'date',
            'symbol',
            F.col('sellerAccountId').alias('accountId'),
            (-F.col('nTradeShares')).alias('nTradeShares'),
            F.col('tradeSettlementValue').alias('settlementValue')
        )
)

In [ ]:
trade_kpi_df = (
    buy_trade_df
    .union(sell_trade_df)
    .groupBy('accountId')
    .agg(
        F.count(F.lit(1)).alias('tradeFrequency'),
        F.mean(F.abs('settlementValue')).alias('meanTradeValue'),
        F.sum('settlementValue').alias('netSumTradeValue'),
        F.sum(F.abs('settlementValue')).alias('absSumTradeValue'),
        F.countDistinct('date').alias('activeDays'),
    )
    .join(active_days_df, on = 'accountId')
)

display_df(trade_kpi_df)

In [ ]:
(
    trade_kpi_df
    .agg(
        F.round(F.expr('percentile(tradeFrequency, array(0.25))')[0], 2).alias('25% percentile'),
        F.round(F.expr('percentile(tradeFrequency, array(0.5))')[0], 2).alias('50% percentile'),
        F.round(F.mean('tradeFrequency'), 2).alias('mean'),
        F.round(F.expr('percentile(tradeFrequency, array(0.75))')[0], 2).alias('75% percentile'),
        F.round(F.expr('percentile(tradeFrequency, array(0.9))')[0], 2).alias('90% percentile'),
        F.round(F.expr('percentile(tradeFrequency, array(0.99))')[0], 2).alias('99% percentile'),
        F.round(F.expr('percentile(tradeFrequency, array(0.999))')[0], 2).alias('99.9% percentile'),
    )
    .show()
)

In [ ]:
(
    trade_kpi_df
    .agg(
        F.round(F.expr('percentile(activeDays, array(0.25))')[0], 2).alias('25% percentile'),
        F.round(F.expr('percentile(activeDays, array(0.5))')[0], 2).alias('50% percentile'),
        F.round(F.mean('activeDays'), 2).alias('mean'),
        F.round(F.expr('percentile(activeDays, array(0.75))')[0], 2).alias('75% percentile'),
        F.round(F.expr('percentile(activeDays, array(0.9))')[0], 2).alias('90% percentile'),
        F.round(F.expr('percentile(activeDays, array(0.99))')[0], 2).alias('99% percentile'),
        F.round(F.expr('percentile(activeDays, array(0.999))')[0], 2).alias('99.9% percentile'),
    )
    .show()
)

In [ ]:
(
    trade_kpi_df
    .agg(
        F.round(F.expr('percentile(nBuyDays, array(0.25))')[0], 2).alias('25% percentile'),
        F.round(F.expr('percentile(nBuyDays, array(0.5))')[0], 2).alias('50% percentile'),
        F.round(F.mean('nBuyDays'), 2).alias('mean'),
        F.round(F.expr('percentile(nBuyDays, array(0.75))')[0], 2).alias('75% percentile'),
        F.round(F.expr('percentile(nBuyDays, array(0.9))')[0], 2).alias('90% percentile'),
        F.round(F.expr('percentile(nBuyDays, array(0.99))')[0], 2).alias('99% percentile'),
        F.round(F.expr('percentile(nBuyDays, array(0.999))')[0], 2).alias('99.9% percentile'),
    )
    .show()
)

In [ ]:
(
    trade_kpi_df
    .agg(
        F.round(F.expr('percentile(nSellDays, array(0.25))')[0], 2).alias('25% percentile'),
        F.round(F.expr('percentile(nSellDays, array(0.5))')[0], 2).alias('50% percentile'),
        F.round(F.mean('nSellDays'), 2).alias('mean'),
        F.round(F.expr('percentile(nSellDays, array(0.75))')[0], 2).alias('75% percentile'),
        F.round(F.expr('percentile(nSellDays, array(0.9))')[0], 2).alias('90% percentile'),
        F.round(F.expr('percentile(nSellDays, array(0.99))')[0], 2).alias('99% percentile'),
        F.round(F.expr('percentile(nSellDays, array(0.999))')[0], 2).alias('99.9% percentile'),
    )
    .show()
)

In [ ]:
print(trade_kpi_df.count() - trade_kpi_df.dropna().count())

In [ ]:
trade_output_df = (
    trade_kpi_df
    .join(return_df.select('accountId', 'return').dropDuplicates(), on = ['accountId'])
    .dropna()
    .withColumn('tradeFrequencyDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('tradeFrequency')))
    .withColumn('meanTradeValueDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('meanTradeValue')))
    .withColumn('netSumTradeValueDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('netSumTradeValue')))
    .withColumn('absSumTradeValueDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('absSumTradeValue')))
    .withColumn('activeDaysDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('activeDays')))
    .withColumn('nBuyDaysDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('nBuyDays')))
    .withColumn('nSellDaysDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('nSellDays')))
)

display_df(trade_output_df)

In [ ]:
print(round(trade_output_df.filter(F.col('return') >= 0.35).count() / trade_output_df.count() ,2))

In [ ]:
(
    trade_output_df
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .show()
)

In [ ]:
(
    trade_output_df
    .groupBy('tradeFrequencyDecile')
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .orderBy('tradeFrequencyDecile')
    .show()
)

In [ ]:
trade_output_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'trade_output.parquet')

### identify block holders

In [ ]:
bh_df = (
    daily_portfolio_df
    .select('date', 'symbol', 'accountId', 'heldShares')
    .join(price_df.select('date', 'symbol', 'shrout'), on = ['date', 'symbol'])
    .withColumn('ownership', F.col('heldShares') / F.col('shrout'))
    .filter( (F.col('ownership') >= 0.01) & F.col('ownership').isNotNull() )
    .select('accountId')
    .distinct()
    .withColumn('isBH', F.lit(1))
)

display_df(bh_df)

In [ ]:
bh_output_df = (
    return_df
    .select('accountId', 'return')
    .dropna()
    .join(bh_df, on = 'accountId', how = 'left')
    .fillna(0, 'isBH')
)

display_df(bh_output_df)

In [ ]:
(
    bh_output_df
    .groupBy('isBH')
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianTradeFrequency')
    )
    .show()
)

In [ ]:
bh_output_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'bhOutput.parquet')

### number of stocks within initial portfolio

In [ ]:
n_stocks_within_initial_portfolio_df = (
    portfolio_df
    .groupBy('accountId')
    .agg(
        F.count(F.lit(1)).alias('nStocksWithinInitialPortfolio')
    )
    .dropna()
)

display_df(n_stocks_within_initial_portfolio_df)

In [ ]:
n_stocks_within_initial_portfolio_output_df = (
    return_df
    .select('accountId', 'return')
    .dropna()
    .join(n_stocks_within_initial_portfolio_df, on = 'accountId', how = 'inner')
    .withColumn('nStocksWithinInitialPortfolioDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('nStocksWithinInitialPortfolio')))
)

display_df(n_stocks_within_initial_portfolio_output_df)

In [ ]:
(
    n_stocks_within_initial_portfolio_output_df
    .groupBy('nStocksWithinInitialPortfolioDecile')
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .orderBy('nStocksWithinInitialPortfolioDecile')
    .show()
)

In [ ]:
n_stocks_within_initial_portfolio_output_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'n_initial_portfolio.parquet')

### number of stocks within final portfolio

In [ ]:
n_stocks_within_final_portfolio_df = (
    daily_portfolio_df
    .withColumn('rowNumber', F.row_number().over(Window.partitionBy('accountId', 'symbol').orderBy('date')))
    .withColumn('maxRowNumber', F.max('rowNumber').over(Window.partitionBy('accountId', 'symbol')))
    .filter(F.col('rowNumber') == F.col('maxRowNumber'))
    .filter(F.col('heldShares') > 0)
    .withColumn('date', F.lit(MAX_PRICE_DATE))
    .join(price_df.select('date', 'symbol', 'close_price'), on = ['date', 'symbol'], how = 'left')
    .dropna(subset = ['close_price'])
    .groupBy('accountId')
    .agg(
        F.countDistinct('symbol').alias('nStocksWithinFinalPortfolio')
    )   
)

display_df(n_stocks_within_final_portfolio_df)

In [ ]:
n_stocks_within_final_portfolio_output_df = (
    return_df
    .select('accountId', 'return')
    .dropna()
    .join(n_stocks_within_final_portfolio_df, on = 'accountId', how = 'inner')
    .withColumn('nStocksWithinFinalPortfolioDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy('nStocksWithinFinalPortfolio')))
)

display_df(n_stocks_within_final_portfolio_output_df)

In [ ]:
(
    n_stocks_within_final_portfolio_output_df
    .groupBy('nStocksWithinFinalPortfolioDecile')
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .orderBy('nStocksWithinFinalPortfolioDecile')
    .show()
)

In [ ]:
n_stocks_within_final_portfolio_output_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'n_final_portfolio.parquet')

### turnover

In [ ]:
turnover_df = (
    trade_kpi_df
    .join(final_portfolio_value_df, on =['accountId'], how = 'right')
    
    .fillna(0, subset = ['absSumTradeValue'])
    .withColumn('turnover', F.col('absSumTradeValue') / F.col('finalPortfolioValue'))
    .join(return_df.select('accountId', 'return'), on = 'accountId')
    .withColumn('turnoverDecile', F.ntile(N_QUANTILES).over(Window.partitionBy().orderBy(F.col('turnover'))))
    .select(
        'accountId',
        'turnover',
        'turnoverDecile',
        'return'
    )
)

display_df(turnover_df)

In [ ]:
(
    turnover_df
    .groupBy('turnoverDecile')
    .agg(
        F.round(F.expr('percentile(return, array(0.5))')[0], 3).alias('medianReturn')
    )
    .orderBy('turnoverDecile')
    .show()
)

In [ ]:
turnover_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'turnover.parquet')

### time series of the number of stocks within portfolio

In [ ]:
dates_list = (
    price_df
    .select('date')
    .distinct()
    .orderBy('date')
    .rdd.flatMap(lambda x: x).collect()
)

print(dates_list)

In [ ]:
nStocksWithinPortfolioOfAllInvestors = []
nInvestors = []
from tqdm import tqdm
for date in tqdm(dates_list):
    result = (
        daily_portfolio_df
        .filter(F.col('date') <= date)
        .withColumn('rowNumber', F.row_number().over(Window.partitionBy('accountId', 'symbol').orderBy('date')))
        .withColumn('maxRowNumber', F.max('rowNumber').over(Window.partitionBy('accountId', 'symbol')))
        .filter(F.col('rowNumber') == F.col('maxRowNumber'))
        .filter( (F.col('heldShares') > 0) & (F.col('heldShares').isNotNull()) )
        .groupBy('accountId')
        .agg(
            F.count(F.lit(1)).alias('nStocksWithinPortfolioOfAllInvestors'),
        )
        .agg(
            F.round(F.mean('nStocksWithinPortfolioOfAllInvestors'), 3).alias('nStocksWithinPortfolioOfAllInvestors'),
            F.count(F.lit(1)).alias('nInvestors')
        )
    )
    nStocksWithinPortfolioOfAllInvestors.append(result.collect()[0][0])
    nInvestors.append(result.collect()[0][1])

In [ ]:
n_stocks_df = spark.createDataFrame(
    pd.DataFrame({
        'date' : dates_list,
        'nStocksWithinPortfolioOfAllInvestors' : nStocksWithinPortfolioOfAllInvestors,
        'nInvestors' : nInvestors
    })
)

display_df(n_stocks_df)

In [ ]:
n_stocks_df.write.mode('overwrite').parquet(PATH_OUTPUT + 'mean_number_of_stocks_within_portfolio.parquet')

In [ ]:
initial_ids =  [row['accountId'] for row in portfolio_df.select('accountId').distinct().collect()]
initial_ids = set(initial_ids)

In [ ]:
unique_id_trade = flat_trade_df.dropDuplicates(subset=['accountId','date'])
unique_id_trade.count()
result = {}
for date in dates_list[:3]:
    print(len(initial_ids))
    tempt = unique_id_trade.filter(F.col('date') == date).select('accountId').distinct().collect()
    teades_ids = set([row['accountId'] for row in tempt])
    result[date] = len(teades_ids -initial_ids )
    
    initial_ids =  set.union(initial_ids, teades_ids)